In [ ]:
import pandas as pd
from itertools import product

# --- Dominoes ---
dominoes = [
    ('RR', ('R','R')),
    ('RB', ('R','B')),
    ('RY', ('R','Y')),
    ('BY', ('B','Y')),
    ('BB', ('B','B')),
    ('YY', ('Y','Y')),
]

# --- Build adjacency pairs for 3x3 ---
pairs = []
rows, cols = 3,3
for r in range(rows):
    for c in range(cols):
        i = r*cols + c
        if c+1 < cols:   # right neighbor
            j = r*cols + (c+1)
            pairs.append((i,j))
        if r+1 < rows:   # down neighbor
            j = (r+1)*cols + c
            pairs.append((i,j))

# --- Color merge mapping ---
def merge_colors(s):
    if not s: return 'W'
    if s == {'R'}: return 'R'
    if s == {'B'}: return 'B'
    if s == {'Y'}: return 'Y'
    if s == {'R','B'}: return 'P'   # Purple
    if s == {'R','Y'}: return 'O'   # Orange
    if s == {'B','Y'}: return 'G'   # Green
    return 'W'

# --- Symmetry helpers ---
def transform(layout, kind):
    L = list(layout)
    if kind == "rot90": return tuple([L[6],L[3],L[0],L[7],L[4],L[1],L[8],L[5],L[2]])
    if kind == "rot180": return tuple([L[8],L[7],L[6],L[5],L[4],L[3],L[2],L[1],L[0]])
    if kind == "rot270": return tuple([L[2],L[5],L[8],L[1],L[4],L[7],L[0],L[3],L[6]])
    if kind == "flipH": return tuple([L[2],L[1],L[0],L[5],L[4],L[3],L[8],L[7],L[6]])
    if kind == "flipV": return tuple([L[6],L[7],L[8],L[3],L[4],L[5],L[0],L[1],L[2]])
    if kind == "flipD1": return tuple([L[0],L[3],L[6],L[1],L[4],L[7],L[2],L[5],L[8]])
    if kind == "flipD2": return tuple([L[8],L[5],L[2],L[7],L[4],L[1],L[6],L[3],L[0]])
    return tuple(L)

def canonical(layout):
    variants = []
    base = tuple(layout)
    for kind in ["", "rot90","rot180","rot270","flipH","flipV","flipD1","flipD2"]:
        variants.append(transform(base, kind) if kind else base)
    return min(variants)

# --- Coverage check ---
def check_coverage(counts, allow_center_empty=False):
    if allow_center_empty:
        if counts[4] != 0: return False
        return all(counts[i] > 0 for i in range(9) if i != 4)
    else:
        return all(counts[i] > 0 for i in range(9))

# --- Main enumeration ---
def enumerate_unique_layouts(require_center_empty=False):
    unique_layouts = set()
    pair_indices = list(range(len(pairs)))

    for assign in product(pair_indices, repeat=len(dominoes)):
        counts = [0]*9
        valid = True
        for di, pair_idx in enumerate(assign):
            a,b = pairs[pair_idx]
            counts[a] += 1
            counts[b] += 1
            if counts[a] > 2 or counts[b] > 2:
                valid = False
                break
        if not valid: continue
        if not check_coverage(counts, allow_center_empty=require_center_empty):
            continue
        # build cell sets
        cell_sets = [set() for _ in range(9)]
        for di, pair_idx in enumerate(assign):
            a,b = pairs[pair_idx]
            _, halves = dominoes[di]
            c1,c2 = halves
            cell_sets[a].add(c1)
            cell_sets[b].add(c2)
        labels = [merge_colors(s) for s in cell_sets]
        unique_layouts.add(canonical(labels))

    return unique_layouts

# --- Generate layouts ---
all_filled = enumerate_unique_layouts(require_center_empty=False)
center_empty = enumerate_unique_layouts(require_center_empty=True)

print("Unique outcomes (all filled):", len(all_filled))
print("Unique outcomes (center empty):", len(center_empty))

# --- Save as DataFrame (no trailing ]) ---
def layouts_to_df(layouts):
    colors = ['R','B','Y','P','O','G']
    records = []
    for row in layouts:
        counts = {c: row.count(c) for c in colors}
        counts["S"] = counts["P"] + counts["O"] + counts["G"]  # <-- add secondary total
        record = {f'c{i}': row[i] for i in range(9)}
        record.update(counts)
        records.append(record)
    return pd.DataFrame(records)

df_all = layouts_to_df(all_filled)
df_center = layouts_to_df(center_empty)


Unique outcomes (all filled): 14525
Unique outcomes (center empty): 7196


In [60]:
df_all = df_all[
    (df_all['S'] == 3) & 
    (df_all['R'] < 3) & (df_all['B'] < 3) & (df_all['Y'] < 3) &
    (df_all['P'] < 3) & (df_all['O'] < 3) & (df_all['G'] < 3)
][['c0','c1','c2','c3','c4','c5','c6','c7','c8']]

df_center = df_center[
    (df_center['S'] == 3) & 
    (df_center['R'] < 3) & (df_center['B'] < 3) & (df_center['Y'] < 3) &
    (df_center['P'] < 3) & (df_center['O'] < 3) & (df_center['G'] < 3)
][['c0','c1','c2','c3','c4','c5','c6','c7','c8']]

df_all['c9'] = "a"
df_center['c9'] = "a"
df_all.to_csv("final_unique_outcomes_all_filled.csv", index=False)
df_center.to_csv("final_unique_outcomes_center_empty.csv", index=False)